<a href="https://colab.research.google.com/github/advait1987/tf_gendervoiceclassify/blob/master/Gender_Voice_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
!pip install -q tensorflow==2.0.0-alpha0
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [15]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../tmp"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [0]:
url = 'https://github.com/advait1987/tf_gendervoiceclassify/raw/master/voice.csv'
dataframe = pd.read_csv(url)

## **Explore**

In [17]:
dataframe.head(2)

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,0.0,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,0.0,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male


In [18]:
from sklearn.preprocessing import LabelEncoder
gender_encoder = LabelEncoder()
dataframe['label'] = gender_encoder.fit_transform(dataframe['label'])
dataframe.head(2)

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,0.0,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,1
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,0.0,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,1


In [19]:
print("Total number of samples: {}".format(dataframe.shape[0]))
print("Number of male: {}".format(dataframe[dataframe.label == 1].shape[0]))
print("Number of female: {}".format(dataframe[dataframe.label == 0].shape[0]))

Total number of samples: 3168
Number of male: 1584
Number of female: 1584


Split Data

In [20]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')
print (train.shape)
train.head()

2027 train examples
507 validation examples
634 test examples
(2027, 21)


,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
2261,0.139254,0.083980,0.151920,0.056905,0.212480,0.155575,1.546572,5.868484,0.975814,0.822671,0.209173,0.139254,0.193576,0.034483,0.262295,0.515625,0.007812,3.921875,3.914062,0.142429,0
1713,0.235349,0.061951,0.255031,0.240735,0.266086,0.025351,2.815614,10.445064,0.832678,0.247794,0.260749,0.235349,0.209523,0.017204,0.275862,0.580181,0.015625,5.500000,5.484375,0.133191,0
1943,0.199387,0.035440,0.196046,0.180719,0.226471,0.045752,2.342196,9.294968,0.860738,0.210884,0.184150,0.199387,0.159956,0.027119,0.271186,0.898438,0.007812,5.976562,5.968750,0.189915,0
1105,0.189809,0.057016,0.202868,0.136830,0.241434,0.104604,0.795922,2.627216,0.917883,0.353845,0.123623,0.189809,0.143059,0.048387,0.277457,1.173018,0.023438,5.273438,5.250000,0.199836,1
1654,0.144419,0.086971,0.156016,0.061026,0.222840,0.161815,0.608371,2.844063,0.976355,0.818056,0.195503,0.144419,0.178420,0.016667,0.275862,0.509301,0.007812,3.898438,3.890625,0.151765,0


**Create an input pipeline using tf.data**
This will enable us to use feature columns as a bridge to map from the columns in the Pandas dataframe to features 
used to train the model. If we were working with a very large CSV file (so large that it does not fit into memory), we would use tf.data to read it from disk directly. That is not covered in this tutorial.

In [0]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('label')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [22]:
# Test input pipeline
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['median'])
  print('A batch of targets:', label_batch )

Every feature: ['meanfreq', 'sd', 'median', 'Q25', 'Q75', 'IQR', 'skew', 'kurt', 'sp.ent', 'sfm', 'mode', 'centroid', 'meanfun', 'minfun', 'maxfun', 'meandom', 'mindom', 'maxdom', 'dfrange', 'modindx']
A batch of ages: tf.Tensor([0.13332539 0.23102167 0.18489655 0.18410735 0.21693842], shape=(5,), dtype=float64)
A batch of targets: tf.Tensor([1 0 1 0 0], shape=(5,), dtype=int32)


**Choose which columns to use**

In [0]:
feature_columns = []

# numeric cols
for header in list(feature_batch.keys()):
  feature_columns.append(feature_column.numeric_column(header))

# bucketized cols
#age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
#feature_columns.append(age_buckets)

# indicator cols
#thal = feature_column.categorical_column_with_vocabulary_list(
#      'thal', ['fixed', 'normal', 'reversible'])
#thal_one_hot = feature_column.indicator_column(thal)
#feature_columns.append(thal_one_hot)

# embedding cols
#thal_embedding = feature_column.embedding_column(thal, dimension=8)
#feature_columns.append(thal_embedding)

# crossed cols
#crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
#crossed_feature = feature_column.indicator_column(crossed_feature)
#feature_columns.append(crossed_feature)

**Create feature layer**

In [0]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
#Earlier, we used a small batch size to demonstrate how feature columns worked. We create a new input pipeline with a larger batch size.
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

**Create, compile, and train the model**

In [27]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.95):
      print (logs.get('acc'))
      print("\nReached 95% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()   

model = tf.keras.Sequential([
  feature_layer,
  #tf.keras.layers.Conv1D(5,input_shape=(20,1), kernel_size=12, padding='same', activation='relu'),
  #tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])



model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['acc'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=50,
          verbose = 1,
          callbacks=[callbacks])

#model.summary()

Epoch 1/50
64/64 [==============================] - 2s 25ms/step - loss: 0.7824 - acc: 0.5451 - val_loss: 0.7050 - val_acc: 0.4951
Epoch 2/50
64/64 [==============================] - 1s 21ms/step - loss: 0.7765 - acc: 0.6273 - val_loss: 0.6580 - val_acc: 0.6568
Epoch 3/50
64/64 [==============================] - 1s 21ms/step - loss: 0.6899 - acc: 0.6515 - val_loss: 0.7041 - val_acc: 0.6331
Epoch 4/50
64/64 [==============================] - 1s 21ms/step - loss: 0.6426 - acc: 0.6559 - val_loss: 0.6462 - val_acc: 0.6844
Epoch 5/50
64/64 [==============================] - 1s 21ms/step - loss: 0.6141 - acc: 0.7045 - val_loss: 0.6091 - val_acc: 0.7160
Epoch 6/50
64/64 [==============================] - 1s 21ms/step - loss: 0.6210 - acc: 0.7347 - val_loss: 0.5655 - val_acc: 0.7535
Epoch 7/50
64/64 [==============================] - 1s 21ms/step - loss: 0.6366 - acc: 0.7554 - val_loss: 1.1801 - val_acc: 0.6864
Epoch 8/50
64/64 [==============================] - 1s 21ms/step - loss: 0.8820 - a

**Accuracy**

In [0]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)